In [ ]:
import os.path
import time
from typing import List

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By

# Zadanie 1 (1 pkt)
Wejdź na stronę https://www.pap.pl i sprawdź czy pozwala ona na web scraping i sprawdź czy pozwala ona na scraping sprawdzając plik `robots.txt`.
Wklej tutaj jego treść.
```
User-agent: *
# Directories
Disallow: /wyszukiwanie/
Disallow: /en/search/
Disallow: /core/
Disallow: /profiles/
Disallow: /node/
Disallow: /en/node/
Disallow: /ru/node/
Disallow: /ua/node/
# Files
Disallow: /README.txt
Disallow: /web.config
# Paths (clean URLs)
Disallow: /admin/
Disallow: /comment/
Disallow: /filter/
Disallow: /search/
Disallow: /user/
Disallow: /media/oembed
Disallow: /*/media/oembed
Disallow: /index.php/
```
Strona nie zabrania web scrapingu całkowicie, jednak ogranicza dostęp do pewnych sekcji.
Gdyby web scraping był całkowicie zakazany, plik wyglądał by tak:
```
User-agent: *
Disallow: /
```

# Zadanie 2 (8 pkt)
Stwórz obiekt driver, który połączy się ze stroną PAP, a następnie użyj go żeby wykonać następujące akcje:
1. Zaakceptuj pliki cookies
2. Zmaksymalizuj okno przeglądarki
3. Zmień język strony na angielski
4. Wejdź w sekcję "Business"
5. Z sekcji "Business" ściągnij wszystkie tytuły do listy _titles_
6. Ściągnij wszystkie zdjęcia z tej sekcji
7. Zejdź na dół strony
8. Przejdź na ostatnią stronę i zwróć jej numer (atrybut `text`)

In [ ]:
options = webdriver.FirefoxOptions()
options.add_argument("--incognito")  # okno incognito żeby przy kolejnych uruchomieniach nie mieć zapamiętanych cookies

driver = webdriver.Firefox(options=options)
driver.get("https://www.pap.pl")

# 1
driver.find_element(By.CLASS_NAME, "ok.closeButton").click()

# 2
driver.maximize_window()

# 3
driver.find_element(By.ID, "lang-switch").click()
driver.find_element(By.CLASS_NAME, "choice-en").click()

# 4
driver.find_element(By.XPATH, "//a[@href='/en/business']").click()

# 5
titles: List[str] = []
articles = driver.find_elements(By.CLASS_NAME, "title")
for article in articles:
    titles.append(article.text)
print(titles)

# 6
imgs_out = "data/05/zad6_out/"
if not os.path.exists(imgs_out):
    os.makedirs(imgs_out)

images = driver.find_elements(By.XPATH,
                              "//div[contains(@class, 'newsList')]//div[contains(@class, 'imageWrapper')]//img")
image_urls: List[str] = []
for idx, img in enumerate(images):
    img_url = img.get_attribute("src")
    if img_url:
        try:
            print(f"Downloading {img_url}")
            img_content = requests.get(img_url).content
            img_filename = os.path.join(imgs_out, f"img_{idx}.jpg")
            with open(img_filename, "wb") as f:
                f.write(img_content)
            print(f"Saved to {img_filename}")
        except Exception as e:
            print(f"Got error {e}")

# 7
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# 8
driver.find_element(By.XPATH, "//li[contains(@class, 'pager__item--last')]/a").click()
# Trzeba wybrać element od nowa po przełączeniu strony, bo przed przełączeniem tekst to "Ostatnia >>", a nie numer strony
current_page = driver.find_element(By.XPATH,
                                   "//li[contains(@class, 'pager__item') and contains(@class, 'is-active') and contains(@class, 'active')]")
print(current_page.text)

# zamykamy przeglądarkę na koniec, małe opóźnienie aby zobaczyć ostatni efekt
time.sleep(2)
driver.quit()

# Zadanie 3 (1 pkt)
Wybierz stronę, z której będziesz pobierać dane do projektu końcowego.

Projekt w parze z Maciejem Niszczotą (s32679)

Propozycja tematu nr 1: GTFS feed (mobilitydatabase.org). Eksperyment czy można prognozować opóźnienia jedynie na podstawie statycznych rozkładów jazdy?
Weryfikacja hipotezy z użyciem feedów GTFS RT (real time). W tym przypadku dane są pobierane z api, a nie stricte scrapowane.
Dokumentacja standardu GTFS: https://gtfs.org/

Propozycja tematu nr 2: Prognoza czasów sprzedaży pojazdów (otomoto.pl)

Propozycja tematu nr 3: Prognoza czasów sprzedaży bądź cen mieszkań (otodom.pl). W przypadku prognoz będzie można zweryfikować wyniki z dostępnymi statystykami odnośnie
rynku mieszkaniowego.
